In [121]:
import nasdaqdatalink as quandl
import numpy as np
import pandas as pd
from sklearn import linear_model

In [122]:
quandl.ApiConfig.api_key = 'NRvcyMwNMXZ2ooDSM3nw'
tickers = ['AMZN','WMT', 'HD', 'COST']

In [123]:
prices = quandl.get_table('SHARADAR/SEP', ticker=tickers, date={'gte':'2021-12-31', 'lte':'2022-12-31'})
prices.sort_values(by=["ticker", "date"], ascending=[True, True], inplace=True)
prices = prices.pivot(index='date', columns='ticker', values='closeadj')
prices.head()


ticker,AMZN,COST,HD,WMT
date,,,,
2021-12-31,166.316,560.752,396.530,140.833
2022-01-03,169.995,559.775,390.444,140.794
2022-01-04,167.119,557.325,394.457,138.214
2022-01-05,163.962,543.190,389.106,140.083
2022-01-06,162.862,543.071,387.692,139.694


In [124]:
returns = prices.pct_change().tail(-1)
returns.head()

ticker,AMZN,COST,HD,WMT
date,,,,
2022-01-03,0.022121,-0.001742,-0.015348,-0.000277
2022-01-04,-0.016918,-0.004377,0.010278,-0.018325
2022-01-05,-0.018891,-0.025362,-0.013565,0.013523
2022-01-06,-0.006709,-0.000219,-0.003634,-0.002777
2022-01-07,-0.004292,-0.024772,-0.029944,0.009542


In [125]:
sigma = returns.cov()
sigma

ticker,AMZN,COST,HD,WMT
ticker,,,,
AMZN,0.000995,0.000350,0.000354,0.000161
COST,0.000350,0.000390,0.000255,0.000198
HD,0.000354,0.000255,0.000389,0.000137
WMT,0.000161,0.000198,0.000137,0.000283


In [126]:
fundamentals = quandl.get_table('SHARADAR/SF1', datekey={'gte':'2022-09-30','lte':'2022-12-31'}, dimension="ART", ticker=tickers)
fundamentals = fundamentals.drop_duplicates("ticker", keep="first")
fundamentals.set_index("ticker", inplace=True)
fundamentals["weight"] = fundamentals["marketcap"] / sum(fundamentals["marketcap"])
fundamentals.sort_values(by=["ticker"], ascending=[True], inplace=True)
fundamentals["weight"]

ticker
AMZN    0.526884
COST    0.101174
HD      0.165371
WMT     0.206572
Name: weight, dtype: float64

In [127]:
returns["MKT"] = (returns * fundamentals["weight"]).sum(axis=1)
returns.head()

ticker,AMZN,COST,HD,WMT,MKT
date,,,,,
2022-01-03,0.022121,-0.001742,-0.015348,-0.000277,0.008883
2022-01-04,-0.016918,-0.004377,0.010278,-0.018325,-0.011442
2022-01-05,-0.018891,-0.025362,-0.013565,0.013523,-0.011969
2022-01-06,-0.006709,-0.000219,-0.003634,-0.002777,-0.004732
2022-01-07,-0.004292,-0.024772,-0.029944,0.009542,-0.007748


In [128]:
lamb = 100
tau = 0.025
pi = np.matmul(lamb * sigma, fundamentals["weight"])
pi

ticker
AMZN    0.065134
COST    0.030661
HD      0.030497
WMT     0.018621
dtype: float64

In [129]:
# equal weighting scheme
P_eq = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

In [130]:
Q = np.array([10, 5, -5, -10]) / 100

In [131]:
# When the covariance matrix of the error term (Ω) is calculated using this method,
# the actual value of the scalar (τ) becomes irrelevant because only the ratio ω / τ enters the model.
omega = np.array([[np.matmul(np.matmul(P_eq[0], sigma), P_eq[0]) * tau, 0, 0, 0],
                  [0, np.matmul(np.matmul(P_eq[1], sigma), P_eq[1]) * tau, 0, 0],
                  [0, 0, np.matmul(np.matmul(P_eq[2], sigma), P_eq[2]) * tau, 0],
                  [0, 0, 0, np.matmul(np.matmul(P_eq[2], sigma), P_eq[2]) * tau]])
omega

array([[2.48676219e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.76189461e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 9.73251538e-06, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.73251538e-06]])

In [132]:
e_r = np.matmul(np.linalg.inv(np.linalg.inv(tau * sigma) + np.matmul(np.matmul(P_eq.T, np.linalg.inv(omega)), P_eq)),
          np.matmul(np.linalg.inv(tau * sigma), pi.T) + np.matmul(np.matmul(P_eq.T, np.linalg.inv(omega)), Q.T))
e_r

array([ 0.06294231,  0.01315989, -0.00411028, -0.02641884])